# Part C - Tuning with GridSearch

This notebook addresses questions 8-11 from the assignment:

8. **GridSearch**: Use gridsearch to identify the best hyperparameters for the 4 models
9. **Performance Estimation**: Fairly estimate the performance of the tuned models
10. **Training Curve**: Include a training curve for SGD regressor convergence
11. **Comparison**: Compare performance before and after hyperparameter tuning


In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Enable experimental features BEFORE importing IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

import warnings
warnings.filterwarnings('ignore')

# Set random state for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("All imports successful!")


In [ ]:
# Load data and create best pipeline from Part B
print("Loading data and setting up best pipeline from Part B...")

# Load training data
data = pd.read_csv('health_insurance_train.csv')
X = data.drop('whrswk', axis=1)
y = data['whrswk']

print(f"Dataset shape: {data.shape}")
print(f"Target variable (whrswk) - Mean: {y.mean():.2f}, Std: {y.std():.2f}")

# Define feature types
numerical_feats = ['experience', 'kidslt6', 'kids618', 'husby']
categorical_feats = ['hhi', 'whi', 'hhi2', 'education', 'race', 'hispanic', 'region']

print(f"\nFeature types defined:")
print(f"  Numerical: {len(numerical_feats)} features")
print(f"  Categorical: {len(categorical_feats)} features")


In [ ]:
# Based on Part B results, we use the best pipeline
# NOTE: Update this based on your Part B results!
# For now, we'll implement both pipelines and use the best one

def create_pipeline1():
    """Pipeline 1: Basic preprocessing"""
    numerical_transf = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy="median")), 
        ('scaler', StandardScaler())
    ])
    
    categorical_transf = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy="most_frequent")), 
        ('encoder', OneHotEncoder())
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transf, numerical_feats),
            ('cat', categorical_transf, categorical_feats)
        ]
    )
    
    return preprocessor

# Assuming Pipeline 1 was best from Part B (update based on your results)
best_pipeline = create_pipeline1()
X_for_training = X

print("✅ Best pipeline from Part B loaded")
print("   Using: Pipeline 1 (Basic preprocessing)")
print("\nNote: If Pipeline 2 was better in your Part B, update this cell accordingly")


In [ ]:
# Get baseline performance from Part B for comparison
print("Calculating baseline performance (default hyperparameters)...")

# Define models with default hyperparameters
models_default = {
    'KNN': KNeighborsRegressor(),
    'SGD': SGDRegressor(random_state=RANDOM_STATE, max_iter=1000),
    'Random Forest': RandomForestRegressor(random_state=RANDOM_STATE),
    'Decision Tree': DecisionTreeRegressor(random_state=RANDOM_STATE)
}

# Store baseline results
baseline_results = {}

for name, model in models_default.items():
    # Create full pipeline
    full_pipeline = Pipeline([
        ('preprocessor', best_pipeline),
        ('regressor', model)
    ])
    
    # Cross-validation
    cv_scores = cross_val_score(full_pipeline, X_for_training, y, cv=5, scoring='neg_mean_absolute_error')
    mae = -cv_scores.mean()
    std = cv_scores.std()
    
    baseline_results[name] = {
        'MAE': mae,
        'std': std
    }
    
    print(f"  {name:<15} MAE: {mae:.4f} (+/- {std * 2:.4f})")

print("\n✅ Baseline performance calculated")


## Question 8: GridSearch for Hyperparameter Tuning

**Approach:**
- Use **systematic hyperparameter search** with GridSearchCV
- Use **logarithmic scales** for parameters that can vary widely (as recommended in assignment hints)
- Use **3-fold cross-validation** during grid search (balance between accuracy and computation time)
- Use **reproducible settings** with fixed random_state

**Hyperparameter Search Ranges:**

### KNN Regressor:
- `n_neighbors`: [3, 5, 7, 9, 11, 15, 20] - test different neighborhood sizes
- `weights`: ['uniform', 'distance'] - uniform vs distance-weighted
- `p`: [1, 2] - Manhattan (p=1) vs Euclidean (p=2) distance

### SGD Regressor:
- `alpha`: [0.0001, 0.001, 0.01, 0.1, 1.0] - regularization strength (logarithmic scale)
- `learning_rate`: ['constant', 'optimal', 'invscaling', 'adaptive'] - learning rate schedules
- `eta0`: [0.001, 0.01, 0.1, 1.0] - initial learning rate (logarithmic scale)
- `max_iter`: [1000, 2000, 5000] - maximum iterations for convergence
- `penalty`: ['l2', 'l1', 'elasticnet'] - regularization type

### Random Forest:
- `n_estimators`: [50, 100, 200, 300] - number of trees
- `max_depth`: [None, 10, 20, 30] - maximum tree depth
- `min_samples_split`: [2, 5, 10] - minimum samples to split
- `min_samples_leaf`: [1, 2, 4] - minimum samples per leaf
- `max_features`: ['sqrt', 'log2', None] - features to consider for splits

### Decision Tree:
- `max_depth`: [None, 5, 10, 15, 20, 25] - maximum tree depth
- `min_samples_split`: [2, 5, 10, 20] - minimum samples to split
- `min_samples_leaf`: [1, 2, 4, 8] - minimum samples per leaf
- `criterion`: ['squared_error', 'friedman_mse', 'absolute_error'] - split quality measure
- `splitter`: ['best', 'random'] - split strategy


In [ ]:
# Question 8: Perform GridSearch for all models
import time

print("Question 8: Hyperparameter Tuning with GridSearchCV")
print("="*70)
print("This will take several minutes...")

# Define parameter grids
param_grids = {
    'KNN': {
        'regressor__n_neighbors': [3, 5, 7, 9, 11, 15, 20],
        'regressor__weights': ['uniform', 'distance'],
        'regressor__p': [1, 2]
    },
    'SGD': {
        'regressor__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],  # Logarithmic scale
        'regressor__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
        'regressor__eta0': [0.001, 0.01, 0.1, 1.0],  # Logarithmic scale
        'regressor__max_iter': [1000, 2000, 5000],
        'regressor__penalty': ['l2', 'l1', 'elasticnet']
    },
    'Random Forest': {
        'regressor__n_estimators': [50, 100, 200, 300],
        'regressor__max_depth': [None, 10, 20, 30],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 4],
        'regressor__max_features': ['sqrt', 'log2', None]
    },
    'Decision Tree': {
        'regressor__max_depth': [None, 5, 10, 15, 20, 25],
        'regressor__min_samples_split': [2, 5, 10, 20],
        'regressor__min_samples_leaf': [1, 2, 4, 8],
        'regressor__criterion': ['squared_error', 'friedman_mse', 'absolute_error'],
        'regressor__splitter': ['best', 'random']
    }
}

# Store tuned models and results
tuned_models = {}
tuning_results = {}

print("\nPerforming GridSearch for each model...")
for name, param_grid in param_grids.items():
    print(f"\n{'='*70}")
    print(f"Tuning {name}...")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    # Create base model
    if name == 'KNN':
        base_model = KNeighborsRegressor()
    elif name == 'SGD':
        base_model = SGDRegressor(random_state=RANDOM_STATE)
    elif name == 'Random Forest':
        base_model = RandomForestRegressor(random_state=RANDOM_STATE)
    elif name == 'Decision Tree':
        base_model = DecisionTreeRegressor(random_state=RANDOM_STATE)
    
    # Create pipeline
    model_pipeline = Pipeline([
        ('preprocessor', best_pipeline),
        ('regressor', base_model)
    ])
    
    # Calculate total combinations
    total_combinations = 1
    for param_values in param_grid.values():
        total_combinations *= len(param_values)
    
    print(f"Testing {total_combinations} parameter combinations with 3-fold CV")
    print(f"Total fits: {total_combinations * 3}")
    
    # Grid search with 3-fold CV
    grid_search = GridSearchCV(
        model_pipeline,
        param_grid,
        cv=3,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,  # Use all available cores
        verbose=1,
        return_train_score=True
    )
    
    # Fit the grid search
    grid_search.fit(X_for_training, y)
    
    # Store results
    tuned_models[name] = grid_search.best_estimator_
    tuning_results[name] = {
        'best_params': grid_search.best_params_,
        'best_score': -grid_search.best_score_,  # Convert back to positive MAE
        'tuning_time': time.time() - start_time,
        'cv_results': grid_search.cv_results_
    }
    
    print(f"\n✅ {name} tuning completed in {tuning_results[name]['tuning_time']:.2f} seconds")
    print(f"Best MAE (CV): {tuning_results[name]['best_score']:.4f}")
    print(f"Best parameters:")
    for param, value in tuning_results[name]['best_params'].items():
        print(f"  {param}: {value}")

print("\n" + "="*70)
print("ALL HYPERPARAMETER TUNING COMPLETED")
print("="*70)


## Question 9: Fair Performance Estimation of Tuned Models

**Why this is a fair estimate:**
1. **5-fold Cross-Validation** on the full dataset - provides robust performance estimate
2. **Separate from tuning** - we use the best model from GridSearch and evaluate it independently
3. **Multiple folds** - reduces variance in the estimate
4. **Same evaluation method** as baseline - allows fair comparison

**How we estimate performance:**
- Use the best model from GridSearch
- Apply 5-fold cross-validation on the full training dataset
- Calculate mean and standard deviation of MAE across folds
- This gives us an unbiased estimate of performance on unseen data


In [ ]:
# Question 9: Evaluate tuned models with 5-fold CV
print("Question 9: Fair Performance Estimation of Tuned Models")
print("="*70)

tuned_performance = {}

for name, model in tuned_models.items():
    print(f"\nEvaluating {name}...")
    
    # 5-fold cross-validation for fair estimate
    cv_scores = cross_val_score(model, X_for_training, y, cv=5, scoring='neg_mean_absolute_error')
    mae = -cv_scores.mean()
    std = cv_scores.std()
    
    tuned_performance[name] = {
        'MAE': mae,
        'std': std,
        'cv_scores': cv_scores
    }
    
    print(f"  MAE: {mae:.4f} (+/- {std * 2:.4f})")
    print(f"  Individual CV scores: {[-score for score in cv_scores]}")

print("\n✅ All tuned models evaluated with 5-fold CV")


## Question 10: Training Curve for SGD Regressor

**Purpose:** Demonstrate that the SGD regressor converges to a reasonable solution

**What to show:**
- Plot MAE vs epochs to show convergence
- Verify that the loss decreases and stabilizes
- Ensure enough iterations were used for convergence


In [ ]:
# Question 10: Create training curve for SGD Regressor
print("Question 10: SGD Regressor Training Curve")
print("="*70)

# Get the best SGD model
best_sgd = tuned_models['SGD']
sgd_regressor = best_sgd.named_steps['regressor']

print(f"Best SGD parameters:")
for param, value in tuning_results['SGD']['best_params'].items():
    print(f"  {param}: {value}")

# Train SGD with verbose output to track convergence
print("\nTraining SGD to track convergence...")

# Create a new SGD model with the best parameters and verbose output
sgd_params = {k.replace('regressor__', ''): v for k, v in tuning_results['SGD']['best_params'].items()}
sgd_for_curve = SGDRegressor(**sgd_params, random_state=RANDOM_STATE, verbose=0)

# Manually track training progress
from sklearn.model_selection import train_test_split

# Preprocess the data first
X_preprocessed = best_pipeline.fit_transform(X_for_training)
X_train, X_val, y_train, y_val = train_test_split(X_preprocessed, y, test_size=0.2, random_state=RANDOM_STATE)

# Train with partial_fit to track progress
max_iter = sgd_params.get('max_iter', 1000)
train_errors = []
val_errors = []
epochs = []

# Initialize the model
sgd_for_curve.partial_fit(X_train[:100], y_train[:100])

# Train epoch by epoch
for epoch in range(1, max_iter + 1):
    # Train on full training set
    sgd_for_curve.partial_fit(X_train, y_train)
    
    # Track errors every 50 epochs
    if epoch % 50 == 0 or epoch == 1:
        train_pred = sgd_for_curve.predict(X_train)
        val_pred = sgd_for_curve.predict(X_val)
        
        train_mae = mean_absolute_error(y_train, train_pred)
        val_mae = mean_absolute_error(y_val, val_pred)
        
        train_errors.append(train_mae)
        val_errors.append(val_mae)
        epochs.append(epoch)
        
        if epoch % 200 == 0:
            print(f"  Epoch {epoch}: Train MAE = {train_mae:.4f}, Val MAE = {val_mae:.4f}")

# Plot training curve
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_errors, 'b-', label='Training MAE', linewidth=2)
plt.plot(epochs, val_errors, 'r-', label='Validation MAE', linewidth=2)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('MAE', fontsize=12)
plt.title('SGD Regressor: Training Curve', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Zoom in on later epochs to show convergence
plt.subplot(1, 2, 2)
if len(epochs) > 10:
    start_idx = len(epochs) // 2
    plt.plot(epochs[start_idx:], train_errors[start_idx:], 'b-', label='Training MAE', linewidth=2)
    plt.plot(epochs[start_idx:], val_errors[start_idx:], 'r-', label='Validation MAE', linewidth=2)
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('MAE', fontsize=12)
    plt.title('SGD Regressor: Convergence (Second Half)', fontsize=14, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Training curve shows SGD converged")
print(f"Final Training MAE: {train_errors[-1]:.4f}")
print(f"Final Validation MAE: {val_errors[-1]:.4f}")
print(f"Convergence achieved: {'Yes' if abs(val_errors[-1] - val_errors[-2]) < 0.01 else 'Check if more iterations needed'}")


## Question 11: Compare Performance Before and After Hyperparameter Tuning

**Comparison approach:**
- Show side-by-side comparison of MAE before and after tuning
- Calculate improvement percentage for each model
- Identify which models benefited most from tuning
- Determine the best overall model after tuning


In [ ]:
# Question 11: Compare performance before and after tuning
print("Question 11: Performance Comparison - Before vs After Tuning")
print("="*80)

# Create comparison table
print(f"\n{'Model':<15} | {'Before (MAE)':<15} | {'After (MAE)':<15} | {'Improvement':<15} | {'Status':<10}")
print("-" * 80)

improvements = {}

for name in baseline_results.keys():
    before_mae = baseline_results[name]['MAE']
    after_mae = tuned_performance[name]['MAE']
    
    improvement = ((before_mae - after_mae) / before_mae * 100)
    improvements[name] = improvement
    
    status = "✅ Better" if improvement > 0 else "⚠️ Worse"
    
    print(f"{name:<15} | {before_mae:<15.4f} | {after_mae:<15.4f} | {improvement:>+13.2f}% | {status:<10}")

# Summary statistics
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

best_improvement_model = max(improvements.keys(), key=lambda x: improvements[x])
worst_improvement_model = min(improvements.keys(), key=lambda x: improvements[x])

print(f"\nMost improved model: {best_improvement_model} ({improvements[best_improvement_model]:+.2f}%)")
print(f"Least improved model: {worst_improvement_model} ({improvements[worst_improvement_model]:+.2f}%)")

avg_improvement = np.mean(list(improvements.values()))
print(f"Average improvement: {avg_improvement:+.2f}%")

# Find best overall model
best_overall_model = min(tuned_performance.keys(), key=lambda x: tuned_performance[x]['MAE'])
best_overall_mae = tuned_performance[best_overall_model]['MAE']

print(f"\n🏆 Best overall model after tuning: {best_overall_model}")
print(f"   MAE: {best_overall_mae:.4f}")
print(f"   Improvement over default: {improvements[best_overall_model]:+.2f}%")

# Visualize the comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart comparing before and after
models = list(baseline_results.keys())
before_maes = [baseline_results[m]['MAE'] for m in models]
after_maes = [tuned_performance[m]['MAE'] for m in models]

x = np.arange(len(models))
width = 0.35

bars1 = ax1.bar(x - width/2, before_maes, width, label='Before Tuning', alpha=0.8, color='lightcoral')
bars2 = ax1.bar(x + width/2, after_maes, width, label='After Tuning', alpha=0.8, color='lightgreen')

ax1.set_xlabel('Model', fontsize=12, fontweight='bold')
ax1.set_ylabel('MAE', fontsize=12, fontweight='bold')
ax1.set_title('Model Performance: Before vs After Hyperparameter Tuning', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(models, rotation=45, ha='right')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

# Improvement percentage chart
improvement_values = [improvements[m] for m in models]
colors = ['green' if imp > 0 else 'red' for imp in improvement_values]

bars3 = ax2.barh(models, improvement_values, color=colors, alpha=0.7)
ax2.set_xlabel('Improvement (%)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Model', fontsize=12, fontweight='bold')
ax2.set_title('Percentage Improvement from Hyperparameter Tuning', fontsize=14, fontweight='bold')
ax2.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax2.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, (bar, val) in enumerate(zip(bars3, improvement_values)):
    ax2.text(val, i, f' {val:+.2f}%', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Comparison complete!")


In [ ]:
# Final model selection and autograder submission
print("Final Model Selection for Autograder")
print("="*70)

# Select best model
final_model = tuned_models[best_overall_model]
final_mae_estimate = tuned_performance[best_overall_model]['MAE']

print(f"Selected model: {best_overall_model}")
print(f"Estimated MAE on unseen data: {final_mae_estimate:.4f}")

# Load autograder data
print("\nLoading autograder data...")
data_autograder = pd.read_csv('health_insurance_autograde.csv')
print(f"Autograder data shape: {data_autograder.shape}")

# Make predictions
print("Making predictions...")
predictions = final_model.predict(data_autograder)

print(f"\nPrediction statistics:")
print(f"  Min: {predictions.min():.4f}")
print(f"  Max: {predictions.max():.4f}")
print(f"  Mean: {predictions.mean():.4f}")
print(f"  Std: {predictions.std():.4f}")

# Create submission file
estimate_MAE_on_new_data = np.array([final_mae_estimate])
predictions_autograder_data = predictions

result = np.append(estimate_MAE_on_new_data, predictions_autograder_data)
pd.DataFrame(result).to_csv("autograder_submission_partC.txt", index=False, header=False)

print(f"\n✅ Submission file created: 'autograder_submission_partC.txt'")
print(f"File contains {len(result)} values (1 MAE estimate + {len(predictions)} predictions)")

# Verify submission
submission_check = pd.read_csv("autograder_submission_partC.txt", header=None)
print(f"\nVerification:")
print(f"  Submission file shape: {submission_check.shape}")
print(f"  First value (MAE estimate): {submission_check.iloc[0, 0]:.4f}")
print(f"  Sample predictions: {submission_check.iloc[1:6, 0].values}")

print(f"\n🎯 Ready for autograder submission!")
print(f"Best model: {best_overall_model}")
print(f"Expected MAE: {final_mae_estimate:.4f}")
